<a href="https://colab.research.google.com/github/lucarubini/DeepLearning_quick_introduction/blob/main/tutorial/bozza_serie_temporale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Importare le librerie necessarie
import csv
import numpy as np
import tensorflow as tf
import pandas as pd

## **Crea Dati**

In [2]:
#Configurazioni per i dati sintetici

N_lines = 1000
N_products = 13
max_quantity = 90 #massima quantita di prodotti

filename_output = 'data.csv'

In [3]:
# open the file in the write mode
with open(filename_output, 'w', encoding='UTF8') as f:
    # create the csv writer
    writer = csv.writer(f)
    header = [f"prod_{i}" for i in range(N_products)]
    writer.writerow(header)

    # write a row to the csv file
    for _ in range(N_lines):
      row = [np.random.randint(max_quantity) for _ in range(N_products)]
      writer.writerow(row)

## **Importare i Dati**

In [5]:
#Configurations
csv_path = 'data.csv'

In [6]:
df = pd.read_csv(csv_path)

In [7]:
#df.describe().transpose()

In [8]:
column_indices = {name: i for i, name in enumerate(df.columns)}
num_features = df.shape[1]
n = len(df)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

In [9]:
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

In [10]:
#train_df.describe().transpose()

In [11]:
#Classe per generare le finestre di dati
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])
  
  def split_window(self, features):
    inputs = features[:, self.input_slice, :]
    labels = features[:, self.labels_slice, :]
    if self.label_columns is not None:
      labels = tf.stack(
          [labels[:, :, self.column_indices[name]] for name in self.label_columns],
          axis=-1)

    # Slicing doesn't preserve static shape information, so set the shapes
    # manually. This way the `tf.data.Datasets` are easier to inspect.
    inputs.set_shape([None, self.input_width, None])
    labels.set_shape([None, self.label_width, None])

    return inputs, labels

  def make_dataset(self, data):
    data = np.array(data, dtype=np.float32)
    ds = tf.keras.preprocessing.timeseries_dataset_from_array(
        data=data,
        targets=None,
        sequence_length=self.total_window_size,
        sequence_stride=1,
        shuffle=True,
        batch_size=32,)
    ds = ds.map(self.split_window)
    return ds

  @property
  def train(self):
    return self.make_dataset(self.train_df)

  @property
  def val(self):
    return self.make_dataset(self.val_df)

  @property
  def test(self):
    return self.make_dataset(self.test_df)

  @property
  def example(self):
    """Get and cache an example batch of `inputs, labels` for plotting."""
    result = getattr(self, '_example', None)
    if result is None:
      # No example batch was found, so get one from the `.train` dataset
      result = next(iter(self.train))
      # And cache it for next time
      self._example = result
    return result


In [ ]:
w2 = WindowGenerator(input_width=10, label_width=4, shift=4,
                     label_columns=[f"prod_{i}" for i in range(13)])
w2

In [ ]:
# Stack three slices, the length of the total window.
example_window = tf.stack([np.array(train_df[:w2.total_window_size]),
                           np.array(train_df[100:100+w2.total_window_size]),
                           np.array(train_df[200:200+w2.total_window_size])])

example_inputs, example_labels = w2.split_window(example_window)

print('All shapes are: (batch, time, features)')
print(f'Window shape: {example_window.shape}')
print(f'Inputs shape: {example_inputs.shape}')
print(f'Labels shape: {example_labels.shape}')

In [ ]:
w2.train.element_spec

In [ ]:
for example_inputs, example_labels in w2.train.take(1):
  print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
  print(f'Labels shape (batch, time, features): {example_labels.shape}')

In [ ]:
example_labels

## **Crea e Addestra un modello**

In [17]:
MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [ ]:
OUT_STEPS=4
INPUT_STEP=6
multi_window = WindowGenerator(input_width=INPUT_STEP,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               label_columns=[f"prod_{i}" for i in range(13)])
print(multi_window)



In [27]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units].
    # Adding more `lstm_units` just overfits more quickly.
    tf.keras.layers.LSTM(32, return_sequences=False),
    # Shape => [batch, out_steps*features].
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features].
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

In [ ]:
history = compile_and_fit(multi_lstm_model, multi_window)

In [ ]:
eval_val = multi_lstm_model.evaluate(multi_window.val)

In [ ]:
eval_test = multi_lstm_model.evaluate(multi_window.test)

In [40]:
#calcolo singolo
A_example = tf.convert_to_tensor(np.random.rand(1,INPUT_STEP,num_features))

In [ ]:
A_example

In [37]:
out_A = multi_lstm_model(A_example)

In [ ]:
out_A.shape

In [ ]:
out_A[0,0,:]